In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**With Duplicate**

In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Conv3D, MaxPooling3D, Flatten, Dense, LSTM, Dropout,
                                   Concatenate, BatchNormalization, ZeroPadding3D)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.utils import resample
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

# Load and duplicate dataset
data_path = '/content/drive/MyDrive/dataset/preprocessed-data/mahnob_HCI_preprocessed_all.npy'
data = np.load(data_path, allow_pickle=True)

def duplicate_dataset(data, num_duplicates):
    duplicated_data = data.tolist()
    for _ in range(num_duplicates):
        duplicated_data.extend(data.tolist())
    return np.array(duplicated_data)

# Duplicate dataset
num_duplicates = 2
data = duplicate_dataset(data, num_duplicates)

# Extract features and labels
X_eeg = np.array([entry['eeg_data'] for entry in data])
X_video = np.array([entry['video_data'] for entry in data])
X_ecg = np.array([entry['ecg_data'] for entry in data])
y = np.array([entry['labels'] for entry in data])

print(f"Number of samples after duplication: {len(data)}")

# Pad video data if needed
if X_video.shape[1:] != (32, 64, 64):
    padded_video = []
    for video in X_video:
        pad_shape = [
            (0, max(0, 32 - video.shape[0])),
            (0, max(0, 64 - video.shape[1])),
            (0, max(0, 64 - video.shape[2]))
        ]
        padded_video.append(np.pad(video, pad_shape, mode='constant'))
    X_video = np.array(padded_video)

# Modified balance_classes function for three modalities
def balance_classes(X1, X2, X3, y, class_idx):
    labels = y[:, class_idx]
    unique_classes, class_counts = np.unique(labels, return_counts=True)
    max_count = np.max(class_counts)

    balanced_X1, balanced_X2, balanced_X3, balanced_y = [], [], [], []
    for cls in unique_classes:
        cls_mask = labels == cls
        cls_X1, cls_X2, cls_X3, cls_y = X1[cls_mask], X2[cls_mask], X3[cls_mask], y[cls_mask]

        if len(cls_y) < max_count:
            X1_oversampled, X2_oversampled, X3_oversampled, y_oversampled = resample(
                cls_X1, cls_X2, cls_X3, cls_y,
                replace=True,
                n_samples=max_count - len(cls_y),
                random_state=42
            )
            cls_X1 = np.concatenate([cls_X1, X1_oversampled])
            cls_X2 = np.concatenate([cls_X2, X2_oversampled])
            cls_X3 = np.concatenate([cls_X3, X3_oversampled])
            cls_y = np.concatenate([cls_y, y_oversampled])

        balanced_X1.append(cls_X1)
        balanced_X2.append(cls_X2)
        balanced_X3.append(cls_X3)
        balanced_y.append(cls_y)

    return (np.concatenate(balanced_X1),
            np.concatenate(balanced_X2),
            np.concatenate(balanced_X3),
            np.concatenate(balanced_y))

# Balance classes
X_eeg, X_video, X_ecg, y = balance_classes(X_eeg, X_video, X_ecg, y, class_idx=1)
X_eeg, X_video, X_ecg, y = balance_classes(X_eeg, X_video, X_ecg, y, class_idx=2)

# Split data
X_eeg_train, X_eeg_test, X_video_train, X_video_test, X_ecg_train, X_ecg_test, y_train, y_test = train_test_split(
    X_eeg, X_video, X_ecg, y, test_size=0.2, random_state=42
)

# Prepare labels
num_valence_classes = int(np.max(y[:, 1])) + 1
num_arousal_classes = int(np.max(y[:, 2])) + 1

y_valence_train = to_categorical(y_train[:, 1], num_classes=num_valence_classes)
y_arousal_train = to_categorical(y_train[:, 2], num_classes=num_arousal_classes)
y_valence_test = to_categorical(y_test[:, 1], num_classes=num_valence_classes)
y_arousal_test = to_categorical(y_test[:, 2], num_classes=num_arousal_classes)

# Model Definition
# 1. Video Branch
video_input = Input(shape=(32, 64, 64, 1), name="video_input")
cnn = ZeroPadding3D(padding=(1, 1, 1))(video_input)
cnn = Conv3D(32, (3, 3, 3), activation='relu')(cnn)
cnn = MaxPooling3D((2, 2, 2))(cnn)
cnn = BatchNormalization()(cnn)
cnn = Conv3D(64, (3, 3, 3), activation='relu')(cnn)
cnn = MaxPooling3D((2, 2, 2))(cnn)
cnn = BatchNormalization()(cnn)
cnn = Flatten()(cnn)
cnn_output = Dense(128, activation='relu')(cnn)

# 2. EEG Branch
eeg_input = Input(shape=(5, 512), name="eeg_input")
lstm_eeg = LSTM(128, return_sequences=True)(eeg_input)
lstm_eeg = Dropout(0.2)(lstm_eeg)
lstm_eeg = LSTM(64)(lstm_eeg)
lstm_eeg_output = Dense(128, activation='relu')(lstm_eeg)

# 3. ECG Branch
ecg_input = Input(shape=(2, 512), name="ecg_input")
lstm_ecg = LSTM(64, return_sequences=True)(ecg_input)
lstm_ecg = Dropout(0.2)(lstm_ecg)
lstm_ecg = LSTM(32)(lstm_ecg)
lstm_ecg_output = Dense(64, activation='relu')(lstm_ecg)

# Combine all modalities
combined = Concatenate()([cnn_output, lstm_eeg_output, lstm_ecg_output])
combined = Dense(256, activation='relu')(combined)
combined = Dropout(0.3)(combined)
combined = Dense(128, activation='relu')(combined)
combined = Dropout(0.2)(combined)

# Output layers
valence_output = Dense(num_valence_classes, activation='softmax', name="valence_output")(combined)
arousal_output = Dense(num_arousal_classes, activation='softmax', name="arousal_output")(combined)

# Create and compile model
model = Model(
    inputs=[video_input, eeg_input, ecg_input],
    outputs=[valence_output, arousal_output]
)

model.compile(
    optimizer='adam',
    loss=['categorical_crossentropy', 'categorical_crossentropy'],
    metrics=['accuracy','accuracy']
)

model.summary()

# Train model
history = model.fit(
    [X_video_train[..., np.newaxis], X_eeg_train, X_ecg_train],
    [y_valence_train, y_arousal_train],
    validation_data=(
        [X_video_test[..., np.newaxis], X_eeg_test, X_ecg_test],
        [y_valence_test, y_arousal_test]
    ),
    epochs=50,
    batch_size=128
)

# Evaluate model
y_valence_pred, y_arousal_pred = model.predict([X_video_test[..., np.newaxis], X_eeg_test, X_ecg_test])

# Calculate metrics
y_valence_pred_classes = np.argmax(y_valence_pred, axis=1)
y_arousal_pred_classes = np.argmax(y_arousal_pred, axis=1)
y_valence_true = np.argmax(y_valence_test, axis=1)
y_arousal_true = np.argmax(y_arousal_test, axis=1)

valence_accuracy = accuracy_score(y_valence_true, y_valence_pred_classes)
arousal_accuracy = accuracy_score(y_arousal_true, y_arousal_pred_classes)

print(f"Valence Accuracy: {valence_accuracy * 100:.2f}%")
print(f"Arousal Accuracy: {arousal_accuracy * 100:.2f}%")


Number of samples after duplication: 558


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ video_input         │ (None, 32, 64,    │          0 │ -                 │
│ (InputLayer)        │ 64, 1)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding3d      │ (None, 34, 66,    │          0 │ video_input[0][0] │
│ (ZeroPadding3D)     │ 66, 1)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d (Conv3D)     │ (None, 32, 64,    │        896 │ zero_padding3d[0… │
│                     │ 64, 32)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling3d       │ (None, 16, 32,    │          0 │ conv3d[0][0]      │
│ (MaxPooling3D)      │ 32, 32)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 16, 32,    │        128 │ max_pooling3d[0]… │
│ (BatchNormalizatio… │ 32, 32)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_1 (Conv3D)   │ (None, 14, 30,    │     55,360 │ batch_normalizat… │
│                     │ 30, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ eeg_input           │ (None, 5, 512)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ecg_input           │ (None, 2, 512)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling3d_1     │ (None, 7, 15, 15, │          0 │ conv3d_1[0][0]    │
│ (MaxPooling3D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 5, 128)    │    328,192 │ eeg_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_2 (LSTM)       │ (None, 2, 64)     │    147,712 │ ecg_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 7, 15, 15, │        256 │ max_pooling3d_1[… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 5, 128)    │          0 │ lstm[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 2, 64)     │          0 │ lstm_2[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 100800)    │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, 64)        │     49,408 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_3 (LSTM)       │ (None, 32)        │     12,416 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │ 12,902,528 │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 128)       │      8,320 │ lstm_1[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 64)        │      2,112 │ lstm_3[0][0]    

 Total params: 13,624,980 (51.98 MB)

 Trainable params: 13,624,788 (51.97 MB)

 Non-trainable params: 192 (768.00 B)

Epoch 1/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 22s 799ms/step - arousal_output_accuracy: 0.0949 - arousal_output_loss: 7.2794 - loss: 14.0158 - valence_output_accuracy: 0.1549 - valence_output_loss: 6.6362 - val_arousal_output_accuracy: 0.1238 - val_arousal_output_loss: 10.1203 - val_loss: 24.8914 - val_valence_output_accuracy: 0.1393 - val_valence_output_loss: 14.6338
Epoch 2/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 13s 685ms/step - arousal_output_accuracy: 0.1423 - arousal_output_loss: 2.1979 - loss: 4.3563 - valence_output_accuracy: 0.2039 - valence_output_loss: 2.1571 - val_arousal_output_accuracy: 0.1796 - val_arousal_output_loss: 13.5836 - val_loss: 27.4280 - val_valence_output_accuracy: 0.1455 - val_valence_output_loss: 13.7250
Epoch 3/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 7s 626ms/step - arousal_output_accuracy: 0.1788 - arousal_output_loss: 2.1462 - loss: 4.2481 - valence_output_accuracy: 0.2330 - valence_output_loss: 2.0972 - val_arousal_output_accuracy: 0.1331 - val_arousal_output_loss: 11.1200 - val_l

**Without Duplicate**

In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Conv3D, MaxPooling3D, Flatten, Dense, LSTM, Dropout,
                                   Concatenate, BatchNormalization, ZeroPadding3D)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.utils import resample
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

# Load and duplicate dataset
data_path = '/content/drive/MyDrive/dataset/preprocessed-data/mahnob_HCI_preprocessed_all.npy'
data = np.load(data_path, allow_pickle=True)

# Extract features and labels
X_eeg = np.array([entry['eeg_data'] for entry in data])
X_video = np.array([entry['video_data'] for entry in data])
X_ecg = np.array([entry['ecg_data'] for entry in data])
y = np.array([entry['labels'] for entry in data])

print(f"Number of samples after duplication: {len(data)}")

# Pad video data if needed
if X_video.shape[1:] != (32, 64, 64):
    padded_video = []
    for video in X_video:
        pad_shape = [
            (0, max(0, 32 - video.shape[0])),
            (0, max(0, 64 - video.shape[1])),
            (0, max(0, 64 - video.shape[2]))
        ]
        padded_video.append(np.pad(video, pad_shape, mode='constant'))
    X_video = np.array(padded_video)

# Modified balance_classes function for three modalities
def balance_classes(X1, X2, X3, y, class_idx):
    labels = y[:, class_idx]
    unique_classes, class_counts = np.unique(labels, return_counts=True)
    max_count = np.max(class_counts)

    balanced_X1, balanced_X2, balanced_X3, balanced_y = [], [], [], []
    for cls in unique_classes:
        cls_mask = labels == cls
        cls_X1, cls_X2, cls_X3, cls_y = X1[cls_mask], X2[cls_mask], X3[cls_mask], y[cls_mask]

        if len(cls_y) < max_count:
            X1_oversampled, X2_oversampled, X3_oversampled, y_oversampled = resample(
                cls_X1, cls_X2, cls_X3, cls_y,
                replace=True,
                n_samples=max_count - len(cls_y),
                random_state=42
            )
            cls_X1 = np.concatenate([cls_X1, X1_oversampled])
            cls_X2 = np.concatenate([cls_X2, X2_oversampled])
            cls_X3 = np.concatenate([cls_X3, X3_oversampled])
            cls_y = np.concatenate([cls_y, y_oversampled])

        balanced_X1.append(cls_X1)
        balanced_X2.append(cls_X2)
        balanced_X3.append(cls_X3)
        balanced_y.append(cls_y)

    return (np.concatenate(balanced_X1),
            np.concatenate(balanced_X2),
            np.concatenate(balanced_X3),
            np.concatenate(balanced_y))

# Balance classes
X_eeg, X_video, X_ecg, y = balance_classes(X_eeg, X_video, X_ecg, y, class_idx=1)
X_eeg, X_video, X_ecg, y = balance_classes(X_eeg, X_video, X_ecg, y, class_idx=2)

# Split data
X_eeg_train, X_eeg_test, X_video_train, X_video_test, X_ecg_train, X_ecg_test, y_train, y_test = train_test_split(
    X_eeg, X_video, X_ecg, y, test_size=0.2, random_state=42
)

# Prepare labels
num_valence_classes = int(np.max(y[:, 1])) + 1
num_arousal_classes = int(np.max(y[:, 2])) + 1

y_valence_train = to_categorical(y_train[:, 1], num_classes=num_valence_classes)
y_arousal_train = to_categorical(y_train[:, 2], num_classes=num_arousal_classes)
y_valence_test = to_categorical(y_test[:, 1], num_classes=num_valence_classes)
y_arousal_test = to_categorical(y_test[:, 2], num_classes=num_arousal_classes)

# Model Definition
# 1. Video Branch
video_input = Input(shape=(32, 64, 64, 1), name="video_input")
cnn = ZeroPadding3D(padding=(1, 1, 1))(video_input)
cnn = Conv3D(32, (3, 3, 3), activation='relu')(cnn)
cnn = MaxPooling3D((2, 2, 2))(cnn)
cnn = BatchNormalization()(cnn)
cnn = Conv3D(64, (3, 3, 3), activation='relu')(cnn)
cnn = MaxPooling3D((2, 2, 2))(cnn)
cnn = BatchNormalization()(cnn)
cnn = Flatten()(cnn)
cnn_output = Dense(128, activation='relu')(cnn)

# 2. EEG Branch
eeg_input = Input(shape=(5, 512), name="eeg_input")
lstm_eeg = LSTM(128, return_sequences=True)(eeg_input)
lstm_eeg = Dropout(0.2)(lstm_eeg)
lstm_eeg = LSTM(64)(lstm_eeg)
lstm_eeg_output = Dense(128, activation='relu')(lstm_eeg)

# 3. ECG Branch
ecg_input = Input(shape=(2, 512), name="ecg_input")
lstm_ecg = LSTM(64, return_sequences=True)(ecg_input)
lstm_ecg = Dropout(0.2)(lstm_ecg)
lstm_ecg = LSTM(32)(lstm_ecg)
lstm_ecg_output = Dense(64, activation='relu')(lstm_ecg)

# Combine all modalities
combined = Concatenate()([cnn_output, lstm_eeg_output, lstm_ecg_output])
combined = Dense(256, activation='relu')(combined)
combined = Dropout(0.3)(combined)
combined = Dense(128, activation='relu')(combined)
combined = Dropout(0.2)(combined)

# Output layers
valence_output = Dense(num_valence_classes, activation='softmax', name="valence_output")(combined)
arousal_output = Dense(num_arousal_classes, activation='softmax', name="arousal_output")(combined)

# Create and compile model
model = Model(
    inputs=[video_input, eeg_input, ecg_input],
    outputs=[valence_output, arousal_output]
)

model.compile(
    optimizer='adam',
    loss=['categorical_crossentropy', 'categorical_crossentropy'],
    metrics=['accuracy','accuracy']
)

model.summary()

# Train model
history = model.fit(
    [X_video_train[..., np.newaxis], X_eeg_train, X_ecg_train],
    [y_valence_train, y_arousal_train],
    validation_data=(
        [X_video_test[..., np.newaxis], X_eeg_test, X_ecg_test],
        [y_valence_test, y_arousal_test]
    ),
    epochs=50,
    batch_size=128
)

# Evaluate model
y_valence_pred, y_arousal_pred = model.predict([X_video_test[..., np.newaxis], X_eeg_test, X_ecg_test])

# Calculate metrics
y_valence_pred_classes = np.argmax(y_valence_pred, axis=1)
y_arousal_pred_classes = np.argmax(y_arousal_pred, axis=1)
y_valence_true = np.argmax(y_valence_test, axis=1)
y_arousal_true = np.argmax(y_arousal_test, axis=1)

valence_accuracy = accuracy_score(y_valence_true, y_valence_pred_classes)
arousal_accuracy = accuracy_score(y_arousal_true, y_arousal_pred_classes)

print(f"Valence Accuracy: {valence_accuracy * 100:.2f}%")
print(f"Arousal Accuracy: {arousal_accuracy * 100:.2f}%")


Number of samples after duplication: 186


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ video_input         │ (None, 32, 64,    │          0 │ -                 │
│ (InputLayer)        │ 64, 1)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding3d_1    │ (None, 34, 66,    │          0 │ video_input[0][0] │
│ (ZeroPadding3D)     │ 66, 1)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_2 (Conv3D)   │ (None, 32, 64,    │        896 │ zero_padding3d_1… │
│                     │ 64, 32)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling3d_2     │ (None, 16, 32,    │          0 │ conv3d_2[0][0]    │
│ (MaxPooling3D)      │ 32, 32)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 16, 32,    │        128 │ max_pooling3d_2[… │
│ (BatchNormalizatio… │ 32, 32)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_3 (Conv3D)   │ (None, 14, 30,    │     55,360 │ batch_normalizat… │
│                     │ 30, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ eeg_input           │ (None, 5, 512)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ecg_input           │ (None, 2, 512)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling3d_3     │ (None, 7, 15, 15, │          0 │ conv3d_3[0][0]    │
│ (MaxPooling3D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_4 (LSTM)       │ (None, 5, 128)    │    328,192 │ eeg_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_6 (LSTM)       │ (None, 2, 64)     │    147,712 │ ecg_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 7, 15, 15, │        256 │ max_pooling3d_3[… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 5, 128)    │          0 │ lstm_4[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_5 (Dropout) │ (None, 2, 64)     │          0 │ lstm_6[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 100800)    │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_5 (LSTM)       │ (None, 64)        │     49,408 │ dropout_4[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_7 (LSTM)       │ (None, 32)        │     12,416 │ dropout_5[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 128)       │ 12,902,528 │ flatten_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 128)       │      8,320 │ lstm_5[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 64)        │      2,112 │ lstm_7[0][0]    

 Total params: 13,624,980 (51.98 MB)

 Trainable params: 13,624,788 (51.97 MB)

 Non-trainable params: 192 (768.00 B)

Epoch 1/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 15s 3s/step - arousal_output_accuracy: 0.1450 - arousal_output_loss: 6.4190 - loss: 13.3983 - valence_output_accuracy: 0.1091 - valence_output_loss: 6.9966 - val_arousal_output_accuracy: 0.1596 - val_arousal_output_loss: 16.5947 - val_loss: 27.3515 - val_valence_output_accuracy: 0.1809 - val_valence_output_loss: 10.7568
Epoch 2/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 701ms/step - arousal_output_accuracy: 0.1507 - arousal_output_loss: 4.8504 - loss: 9.3921 - valence_output_accuracy: 0.1314 - valence_output_loss: 4.5931 - val_arousal_output_accuracy: 0.1277 - val_arousal_output_loss: 52.4928 - val_loss: 71.8578 - val_valence_output_accuracy: 0.1596 - val_valence_output_loss: 19.3651
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 719ms/step - arousal_output_accuracy: 0.1467 - arousal_output_loss: 3.6780 - loss: 7.2931 - valence_output_accuracy: 0.1485 - valence_output_loss: 3.6016 - val_arousal_output_accuracy: 0.1383 - val_arousal_output_loss: 44.9029 - val_loss: 67.80